In [15]:
# Step 1: Create the data frame
my_data3 <- data.frame(
    t = c(2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014),
    xt = c(-0.56, 0.07, 0.12, 1.71, 0.46, -1.26, -0.68, -0.44, 1.22, 0.35),
    yt = c(22, 13, 27, 15, 10, 6, 18, 9, 17, 12),
    ht = c(14, 13, 25, 15, 17, 19, 22, 20, 16, 14)
)

# View the data frame
print(my_data3)

# Step 2: Load necessary libraries
if (!require(tseries)) install.packages("tseries")
if (!require(vars)) install.packages("vars")
if (!require(urca)) install.packages("urca")
if (!require(lmtest)) install.packages("lmtest")
if (!require(car)) install.packages("car")

library(tseries)
library(vars)
library(urca)
library(lmtest)
library(car)

# Step 3: Perform ADF test for each variable
adf_xt <- adf.test(my_data3$xt)
adf_yt <- adf.test(my_data3$yt)
adf_ht <- adf.test(my_data3$ht)

# Print ADF results
print(adf_xt)
print(adf_yt)
print(adf_ht)

# Step 4: Perform ADF test for first differences (for non-stationary series)
adf_yt1 <- adf.test(diff(my_data3$yt, differences = 1))
adf_ht1 <- adf.test(diff(my_data3$ht, differences = 1))

# Print results of ADF test for differenced series
print(adf_yt1)
print(adf_ht1)

# Step 5: Combine variables into a time series matrix
ts_data <- ts(cbind(my_data3$xt, my_data3$yt, my_data3$ht), start = c(2005), frequency = 1)

# Check the structure of the ts_data matrix
str(ts_data)
dim(ts_data)

# Step 6: Perform Johansen Cointegration Test
johansen_test <- ca.jo(ts_data, type = "trace", K = 2, ecdet = "const")
summary(johansen_test)

# Step 7: Determine the optimal lag length using VARselect
lag_selection <- VARselect(ts_data, lag.max = 3, type = "const")  # Adjust lag.max to a smaller value
print(lag_selection)

# Choose the optimal lag based on AIC (or another criterion)
optimal_lag <- lag_selection$selection["AIC(n)"]

# Ensure the optimal lag is appropriate for the dataset size
if (optimal_lag >= nrow(ts_data)) {
  optimal_lag <- nrow(ts_data) - 1
}

# Step 8: Estimate the VAR model
var_model <- VAR(ts_data, p = optimal_lag, type = "const")
summary(var_model)

# Step 9: Estimate the VECM (if cointegration is found)
vecm_model <- ca.jo(ts_data, type = "eigen", K = 2, ecdet = "const")
summary(vecm_model)

# Step 10: Test for robustness
# Check for autocorrelation using Durbin-Watson Test
dw_test <- dwtest(var_model$varresult[[1]])

# Breusch-Pagan Test for heteroscedasticity
bp_test <- bptest(var_model$varresult[[1]])

# Shapiro-Wilk Test for normality of residuals
shapiro_test <- shapiro.test(residuals(var_model$varresult[[1]]))

# Print p-values of the tests
cat("Durbin-Watson Test p-value: ", dw_test$p.value, "\n")
cat("Breusch-Pagan Test p-value: ", bp_test$p.value, "\n")
cat("Shapiro-Wilk Test p-value: ", shapiro_test$p.value, "\n")

# Step 11: Model stability check (check for explosive roots)
# Check for the stability of the VECM model
vecm_stability_check <- function(model) {
  roots <- roots(model)
  print(roots)
  if (any(Mod(roots) > 1)) {
    print("Model is unstable (roots outside unit circle)")
  } else {
    print("Model is stable (roots inside unit circle)")
  }
}

# Stability check for VECM model
vecm_stability_check(vecm_model)

      t    xt yt ht
1  2005 -0.56 22 14
2  2006  0.07 13 13
3  2007  0.12 27 25
4  2008  1.71 15 15
5  2009  0.46 10 17
6  2010 -1.26  6 19
7  2011 -0.68 18 22
8  2012 -0.44  9 20
9  2013  1.22 17 16
10 2014  0.35 12 14

	Augmented Dickey-Fuller Test

data:  my_data3$xt
Dickey-Fuller = -3.2817, Lag order = 2, p-value = 0.09421
alternative hypothesis: stationary


	Augmented Dickey-Fuller Test

data:  my_data3$yt
Dickey-Fuller = -1.5935, Lag order = 2, p-value = 0.7272
alternative hypothesis: stationary


	Augmented Dickey-Fuller Test

data:  my_data3$ht
Dickey-Fuller = -0.92042, Lag order = 2, p-value = 0.9323
alternative hypothesis: stationary



Warning message in adf.test(diff(my_data3$yt, differences = 1)):
"p-value smaller than printed p-value"
Warning message in adf.test(diff(my_data3$ht, differences = 1)):
"p-value smaller than printed p-value"



	Augmented Dickey-Fuller Test

data:  diff(my_data3$yt, differences = 1)
Dickey-Fuller = -16.081, Lag order = 2, p-value = 0.01
alternative hypothesis: stationary


	Augmented Dickey-Fuller Test

data:  diff(my_data3$ht, differences = 1)
Dickey-Fuller = -5.4865, Lag order = 2, p-value = 0.01
alternative hypothesis: stationary

 Time-Series [1:10, 1:3] from 2005 to 2014: -0.56 0.07 0.12 1.71 0.46 -1.26 -0.68 -0.44 1.22 0.35 ...
 - attr(*, "dimnames")=List of 2
  ..$ : NULL
  ..$ : chr [1:3] "Series 1" "Series 2" "Series 3"


[1] 10  3

Warning message in log(1 - lambda[(x + 1):P]):
"NaNs produced"



###################### 
# Johansen-Procedure # 
###################### 

Test type: trace statistic , without linear trend and constant in cointegration 

Eigenvalues (lambda):
[1] 1.000000e+00 1.000000e+00 5.387050e-01 4.440892e-16

Values of teststatistic and critical values of test:

           test 10pct  5pct  1pct
r <= 2 |   6.19  7.52  9.24 12.97
r <= 1 | 264.64 17.85 19.96 24.60
r = 0  |    NaN 32.00 34.91 41.07

Eigenvectors, normalised to first column:
(These are the cointegration relations)

            Series.1.l2 Series.2.l2 Series.3.l2    constant
Series.1.l2   1.0000000   1.0000000   1.0000000  1.00000000
Series.2.l2  -0.1449186  -0.5006455  -0.5510036 -0.22607148
Series.3.l2   0.1005725  -2.4631689   0.1668712 -0.01349247
constant      0.0881667  52.9503781   4.1456667  1.50216597

Weights W:
(This is the loading matrix)

           Series.1.l2 Series.2.l2 Series.3.l2      constant
Series.1.d   -1.099328 -0.09493591   0.5762887  4.432815e-17
Series.2.d  -10.813829  0.3

$selection
AIC(n)  HQ(n)  SC(n) FPE(n) 
     2      2      2      3 

$criteria
               1    2    3
AIC(n)  3.535519 -Inf -Inf
HQ(n)   2.389450 -Inf -Inf
SC(n)   3.442794 -Inf -Inf
FPE(n) 54.860675  NaN    0



ERROR: Error in solve.default(Sigma): Lapack routine dgesv: system is exactly singular: U[3,3] = 0
